In [1]:
import re
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from openai import OpenAI

## 処理の定義

In [2]:
def extract_think_and_others(text: str):
    pattern = re.compile(r"<think>(.*?)</think>", re.DOTALL)
    results = []
    last_end = 0

    for match in pattern.finditer(text):
        # think前のnon_think部分
        non_think = text[last_end:match.start()]
        if non_think.strip():
            results.append({
                "type": "non_think",
                "content": non_think.strip()
            })

        # think部分
        think = match.group(1)
        results.append({
            "type": "think",
            "content": think.strip()
        })

        last_end = match.end()

    # 最後の残り部分
    if last_end < len(text):
        tail = text[last_end:]
        if tail.strip():
            results.append({
                "type": "non_think",
                "content": tail.strip()
            })
    return results


class OpenAIRequest:
    def __init__(
        self,
        base_url: str='http://vllm:8000/v1',
        chat_args={
            'max_tokens': 8192,
            'temperature': 0.7,
            'top_p': 0.8,
            'presence_penalty': 1.5,
            'extra_body': {
                'top_k': 20,
                'chat_template_kwargs': {'enable_thinking': True}
            },
            'model': '/models/Qwen3-8B-FP8'
        }
    ):
        self.client = OpenAI(
            api_key='EMPTY',
            base_url=base_url
        )
        self.chat_args = chat_args

    def __call__(
        self,
        prompt: str,
    ):
        chat_response = self.client.chat.completions.create(
            messages=[{'role': 'user', 'content': prompt}],
            **self.chat_args
        )
        text = chat_response.choices[0].message.content
        return extract_think_and_others(text)

## OpenAIクライアントを使用してみる

In [3]:
open_ai_request = OpenAIRequest()
result = open_ai_request('英語と日本語どちらが難しい？')
for res in result:
    if res['type'] == 'think':
        continue
    else:
        print(res['content'])

英語と日本語のどちらが難しいかは、個人の経験や背景によって異なりますが、以下のような観点から比較できます。

---

### **1. 文法構造**
- **英語**  
  - **主語-動詞-目的語（SVO）** が基本ですが、文型の多様性（例：受動態、条件文など）や不定詞・分詞などの複雑な構文が学習者を悩ますことがあります。  
  - 語順の柔軟性があり、文脈に依存する表現が多いため、理解に時間がかかる場合があります。  

- **日本語**  
  - **主語-目的語-動詞（SOV）** で、文の構造が一貫しています。  
  - ただし、助詞（「が」「は」「を」など）の使い分けや、敬語・丁寧語の区別が難しいです。また、動詞の活用形（未然形、連用形など）も複雑です。  

**→ 結論**：英語の文法は「例外が多すぎる」一方、日本語は「ルールが複雑すぎる」と感じられるかもしれません。

---

### **2. 書き言葉・読み書き**
- **英語**  
  - アルファベットはシンプルですが、発音とスペルの不一致（例：「knight」→ /naɪt/）や、複数形・所有格の規則性の欠如（例：「child」→ 「children」）が難点です。  
  - 括弧や引用符の使用、大文字小文字のルールも注意が必要です。  

- **日本語**  
  - 漢字（かな・カタカナ・漢字）の3種類の文字体系があり、読解には「熟語の意味」や「仮名の読み方」の知識が不可欠です。  
  - 漢字の読みが「音読み」「訓読み」の両方あることが多く、混乱しやすいです（例：「木」→ き／もく）。  

**→ 結論**：英語は「読みにくい」、日本語は「書くのが難しい」と感じる人が多いです。

---

### **3. 発音・アクセント**
- **英語**  
  - インターナショナル・フェイクス（音声の変化）や、母音・子音の区別が難しいです。  
  - 一部の地域方言（例：アメリカ英語vsイギリス英語）の違いも学習障壁になります。  

- **日本語**  
  - ローカルなアクセント（例：関西弁）や、鼻音・半濁音の区別が難しく、外国人にとって聞き取りにくいです。  
  - 音節の短さと語尾の省略（例：「ありがとう」→ 「うん」）も理解に時間がかかります

## langchainを使用してみる

In [7]:
messages = [
    HumanMessage(
        content='英語と日本語どちらが難しい？'
    )
]
llm = ChatOpenAI(
    openai_api_key='EMPTY',
    openai_api_base='http://vllm:8000/v1',
    model_name='/models/Qwen3-8B-FP8',
)
chain = (
    llm
    | StrOutputParser()
)
original_result = chain.invoke(
    messages,
    max_tokens=8192,
    top_p=0.8,
    temperature=0.7,
    presence_penalty=1.5,
    extra_body={
        'top_k': 20,
        'chat_template_kwargs': {'enable_thinking': False},
    }
)
result = extract_think_and_others(original_result)
for res in result:
    if res['type'] == 'think':
        continue
    else:
        print(res['content'])

英語と日本語のどちらが難しいかは、**学習者の母語や経験、目的**によって大きく異なります。以下にそれぞれの特徴を比較し、難易度の観点から説明します。

---

### 🇯🇵 日本語の難しさ

#### 1. **文法構造**
- 日本語は「主語＋動詞＋目的語」の順序（SVO）ですが、実際には主語を省略することが多く、文脈に依存するため、理解が難しい場合があります。
- また、「です・ます」調や敬語・丁寧語の使い分けも重要で、初心者にとっては混乱しやすいです。

#### 2. **漢字・かな・ひらがな**
- 漢字は約2,000～3,000個あり、読み方や意味が複雑で、覚えるのが大変です。
- かな（カタカナ・ひらがな）の使い分けも理解が必要です。

#### 3. **音声と表記の不一致**
- 日本語の発音と表記（ルビ）は必ずしも一致しません（例：「おはよう」→「Ohayou」）。このギャップは学習者にとって障壁になります。

#### 4. **文化・表現の違い**
- 日本語には多くの慣用句や文化に根ざした表現があり、直訳では意味が伝わらないことがあります。

---

### 🇬🇧 英語の難しさ

#### 1. **文法構造**
- 英語は「主語＋動詞＋目的語」の順序（SVO）ですが、時制や否定、疑問文などの文法が複雑で、特に英語初心者にとっては難しいです。
- 複数形、冠詞（a/an/the）、所有格（'s）なども習得に時間がかかります。

#### 2. **発音と綴りの不一致**
- 英語の発音とスペルは一致しないことが多く、例えば「knight」は「ナイツ」と発音されるなど、学習者が苦労するポイントです。

#### 3. **イディオム・文化の違い**
- 英語にも多くの慣用句や文化的背景があるため、単語だけを覚えても意味が分からないことがあります。

#### 4. **多様なアクセント・口語**
- 英語には地域ごとのアクセントや口語表現があり、理解が難しい場合があります。

---

### 🔍 どちらが難しい？結論

| 項目 | 日本語 | 英語 |
|------|--------|------|
| 文法構造 | 相対的にシンプル | より複雑 |
| 発音・綴り | 不一致が多い | 不一致が多い |
| 漢字・かな | 学